# Formating experimental results

In [ ]:
%matplotlib inline
import os
import sys
import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from models import MTC

In [ ]:
plt.style.use('seaborn')

In [ ]:
datasets = [('aotm2011', 'AotM-2011'), ('30music', '30Music')]
metrics = ['R-Precision', 'AUC', 'Hit-Rate']
TOPs = [5, 10, 20, 30, 50, 100, 200, 300, 500, 1000]

## Playlist augmentation

In [ ]:
algos = [('br2', 'Logistic Regression'), ('lrpop', 'LR Popularity'), ('pop', 'Popularity Rank'),
         ('sagh', 'SAGH'), ('cagh', 'CAGH'), ('pla', 'Multitask Classification'), ('plapop', 'MTC Popularity')]

### Performance table

In [ ]:
mix = 1
metric = metrics[mix]

In [ ]:
#col_indices = pd.MultiIndex.from_product([algo_names, ['K=%d' % k for k in range(1,5)]])
#rows = ['K=%d' % k for k in range(1,5)]
#cols = pd.MultiIndex.from_product([algo_names, metrics])
#rows = pd.MultiIndex.from_product([datasets, algo_names])
#cols = pd.MultiIndex.from_product([metrics, ['K=%d' % k for k in range(1,5)]])
rows = [[a[1] for a in algos]]
cols = pd.MultiIndex.from_product([[d[1] for d in datasets], ['K=%d' % k for k in range(1,5)]])

In [ ]:
df = pd.DataFrame(index=rows, columns=cols)

In [ ]:
for algo in algos:
    row_ix = algo[1]
    for dataset in datasets:
        for k in range(1,5):
            data_dir = 'data/%s/setting2' % dataset[0]
            fperf = os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo[0], k))
            if not os.path.exists(fperf):
                continue
            perf_dict = pkl.load(open(fperf, 'rb'))
            #print(perf_dict)
            col_ix = (dataset[1], 'K=%d' % k)
            df.loc[row_ix, col_ix] = perf_dict[dataset[0]]['Test'][metric] * 100

In [ ]:
df

In [ ]:
tab_str = df.to_latex(float_format=lambda x: '$%.1f$' % x, na_rep='N/A', \
                      column_format='l' + '|*{%d}{c}' % 4 * len(datasets), \
                      multirow=True, multicolumn=True, multicolumn_format='c', escape=False)

In [ ]:
print('\\begin{table}[!h]')
print('\centering')
print('\\caption{Performance in terms of %s ($\\times 10^2$)}' % metric)
print('\\label{tab:perf_pla_%s}' % metric.lower())
#print('\\resizebox{\\linewidth}{!}{')
print('\\resizebox{\\columnwidth}{!}{')
print(tab_str)
print('}')
print('\\end{table}')

### Hit Rate curve

In [ ]:
dix = 1
dataset_name = datasets[dix][0]

In [ ]:
#metric = 'Hit-Rate'
metric = metrics[2]

In [ ]:
nrows, ncols = 1, 4
colors = ['r', 'g', 'b', 'm', 'c', 'y']
linestyles = ['-', '--', ':', '-.', '-', '--']
fig = plt.figure(figsize=[20, 3])
for k in range(1,5):
    data_dir = 'data/%s/setting2' % dataset_name
    fperfs = [os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo, k)) for algo, _ in algos]
    perf_dicts = [pkl.load(open(fperf, 'rb')) if os.path.exists(fperf) else None for fperf in fperfs]
    for j in range(len(perf_dicts)):
        if perf_dicts[j] is None:
            continue
        #print(perf_dicts[j])
        x = TOPs
        y = [perf_dicts[j][dataset_name]['Test'][metric][k] for k in TOPs]
        ax = plt.subplot(nrows, ncols, k)
        ax.plot(x, y, ls=linestyles[j], c=colors[j], label=algos[j][1])
        ax.set_xscale('log')
        ax.set_title('K=%d' % k)
    plt.legend(loc='best')
    plt.xlabel('Number of recommendations')
    if k == 1:
        plt.ylabel('Hit rate')
#plt.suptitle('Hit rates of playlist augmentation (%s)' % dataset_name, fontsize=12)
plt.savefig('%s-pla.svg' % dataset_name)

## New song recommendation

In [ ]:
algo_names = ['Logistic Regression', 'PopTest Rank', 'LR Popularity', 'Shortest First', 'Longest First',
              'Multitask Classification']
algos = ['br1', 'poptest', 'lrpop', 'shortest', 'longest', 'nsr']

### Performance table

In [ ]:
rows = pd.MultiIndex.from_product([datasets, algo_names])
cols = metrics

In [ ]:
df = pd.DataFrame(index=rows, columns=cols)

In [ ]:
for dataset_name in datasets:
    for i in range(len(algos)):
        algo_name = algo_names[i]
        algo = algos[i]
        fperf = os.path.join('data/%s/setting1/perf-%s.pkl' % (dataset_name, algo))
        if not os.path.exists(fperf):
            continue
        perf_dict = pkl.load(open(fperf, 'rb'))
        #print(perf_dict)
        for metric in metrics:
            df.loc[(dataset_name, algo_name), metric] = perf_dict[dataset_name]['Test'][metric] * 100

In [ ]:
df

In [ ]:
tab_str = df.to_latex(float_format=lambda x: '$%.2f$' % x, na_rep='N/A', \
                      column_format='ll|' + '*{%d}{c}' % len(metrics), \
                      multirow=True, multicolumn=True, multicolumn_format='c', escape=False)

In [ ]:
print('\\begin{table}[!h]')
print('\centering')
print('\\caption{Performance of new song recommendation $\\times 10^2$}')
print('\\label{tab:perf_nsr}')
print('\\resizebox{\\columnwidth}{!}{')
print(tab_str)
print('}')
print('\\end{table}')

### Hit Rate curve

In [ ]:
metric = 'Hit-Rate'

In [ ]:
nrows, ncols = 1, 4
colors = ['r', 'g', 'b', 'm', 'c']
fig = plt.figure(figsize=[20, 3])
for k in range(1,5):
    fperfs = [os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo, k)) for algo in algos]
    perf_dicts = [pkl.load(open(fperf, 'rb')) if os.path.exists(fperf) else None for fperf in fperfs]
    for j in range(len(perf_dicts)):
        if perf_dicts[j] is None:
            continue
        x = TOPs
        y = [perf_dicts[j][dataset_name]['Test'][metric][k] for k in TOPs]
        ax = plt.subplot(nrows, ncols, k)
        ax.plot(x, y, ls='-', c=colors[j], label=algo_names[j])
        ax.set_xscale('log')
        ax.set_title('K=%d' % k)
    plt.legend(loc='best')
    plt.xlabel('Number of recommendations')
    if k == 1:
        plt.ylabel('Hit rate')
#plt.suptitle('Hit rates of playlist augmentation (%s)' % dataset_name, fontsize=12)
plt.savefig('%s-%d.svg' % (dataset_name, task))